# Project: Binary Bernoulli Bandit Risk Aware Ticker Picker
In the binary Bernoulli bandit ticker picker example, we evaluated the return of the possible tickers over investment periods of length `buffersize::Int64.` However, we failed to consider the risk associated with these tickers. In this project, students will refactor the example code to include the risk for each ticker.

### Learning objectives and tasks
Fill me in.

## Setup
We set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our lab problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl).

In [3]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CH

## Prerequisites: Load ticker-picker save file
Let's begin by loading the [HDF5 encoded saved file](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) generated in the worked example using [a `load(...)`  method exported by the JLD2.jl package](https://github.com/JuliaIO/JLD2.jl). First, we specify the path to the saved file in the `path_to_save_file::String` variable:

In [5]:
number_of_agents = 5000; # how many agents do we want
path_to_save_file = joinpath(_PATH_TO_DATA, "TickerPickerAgent-Simulation-State-N-$(number_of_agents).jld2");

then we [call the `load(...)` method exported by the JLD2.jl package](https://github.com/JuliaIO/JLD2.jl), which reads the binary saved file and returns the saved data as a dictionary; we assign the data to the `saved_state_dict::Dict{String, Any}` variable:

In [7]:
saved_state_dict = load(path_to_save_file)

Dict{String, Any} with 9 entries:
  "risk_free_rate"              => 0.047
  "maximum_number_trading_days" => 1508
  "buffersize"                  => 5
  "data"                        => Dict{String, DataFrame}("TPR"=>1508×8 DataFr…
  "preferences"                 => Beta[Beta{Float64}(α=2.0, β=6.0) Beta{Float6…
  "tickers"                     => ["MRK", "JNJ", "MET", "NFLX", "AAPL", "AMD",…
  "probability_top_dictionary"  => Dict("MSFT"=>0.5858, "JPM"=>0.3042, "C"=>0.2…
  "banditmodel"                 => MyEpsilonSamplingBanditModel([1.0, 1.0, 1.0,…
  "Δt"                          => 0.00396825

Fill me in

In [9]:
dataset = saved_state_dict["data"]; # OHLC data 460 or so firms/ETFs
my_tickers = saved_state_dict["tickers"] |> sort; # *sorted* list of tickers we specified in the worked example
Δt = saved_state_dict["Δt"]; # time step in the data
maximum_number_trading_days = saved_state_dict["maximum_number_trading_days"]; # how many trading days is in the clean data
risk_free_rate = saved_state_dict["risk_free_rate"]; # what is the risk-free rate we used in the example
risk_blind_probability_top_dictionary = saved_state_dict["probability_top_dictionary"];

index_market = findfirst(x -> x == "SPY", my_tickers); # what is the index of SPY in my_tickers
K = length(my_tickers); # how tickers (arms) do we have? 

## Task 1: Compute the risk measure dictionary for my ticker list
While there are several ways to compute risk, in this project, let's borrow an idea from our earlier studies on portfolio allocation and use $\beta_{i}$, i.e., the covariance of the growth (or return) of ticker $\mu_{i}$ with respect to the market growth $\mu_{m}$, normalized by the variance of the market:
$$
\begin{equation}
\beta_{i}\equiv\frac{\text{cov}({\mu_{i},\mu_{m}})}{\text{var}(\mu_{m})}\quad\forall{i}\in\mathcal{P}
\end{equation}
$$

Fill me in

### TODO: Compute the covariance for firms in the `my_tickers` list
The covariance matrix $\Sigma$ will be used to estimate $\beta$ values for the `tickers`$\in$`my_tickers.` 

* First, we compute the expected (annualized) log growth rate by passing the `dataset::Dict{String, DataFrame}` and the `my_tickers::Array{String,1}` list to the [log_growth_matrix(...) method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/equity/#VLQuantitativeFinancePackage.log_growth_matrix). This will compute the growth rates and store them in the `all_firms_return_matrix` variable, a $T-1\times{N}$ array of log growth values. Each row of `all_firms_return_matrix` corresponds to a time value, while each column corresponds to a firm:

In [12]:
all_firms_return_matrix = log_growth_matrix(dataset, my_tickers, Δt = Δt, 
    risk_free_rate = 0.0); # *not* excess growth, so set risk_free_rate = 0

* Next, we estimate the annualized `covariance_matrix` (assuming `252` trading days per year) from the `all_firms_return_matrix` using the [cov(...) function](https://docs.julialang.org/en/v1/stdlib/Statistics/#Statistics.cov), exported by the [Statistics.jl package](https://docs.julialang.org/en/v1/stdlib/Statistics/). We store the $N\times{N}$ covariance matrix in the $\Sigma$ variable:

In [14]:
Σ = cov(all_firms_return_matrix) |> x -> x*(1/252) # annualized, historical volatility

29×29 Matrix{Float64}:
 0.0691322   0.0594079   0.0302666   …  0.0686136   0.02538    0.0262735
 0.0594079   0.211496    0.0425124      0.0995237   0.0284595  0.036321
 0.0302666   0.0425124   0.0894146      0.0498514   0.0298376  0.0780059
 0.0351969   0.050067    0.0849407      0.0590759   0.0327878  0.080665
 0.0224277   0.0281091   0.0314937      0.0299769   0.0183759  0.0308962
 0.0306041   0.040796    0.0414892   …  0.0451837   0.0265421  0.0396317
 0.0319759   0.0452737   0.0676833      0.0500259   0.0276715  0.0632671
 0.0380052   0.0536885   0.035675       0.0549636   0.0192692  0.0330759
 0.0153932   0.0129632   0.0166998      0.0130472   0.0189617  0.0158655
 0.0269166   0.0354797   0.0715436      0.0429108   0.0277043  0.0667961
 0.00473437  0.00336195  0.00283112  …  0.00182335  0.0033409  0.00247071
 0.0167374   0.0164346   0.0247161      0.0156101   0.0204447  0.0254018
 0.0297858   0.0378785   0.0742538      0.0438053   0.0323739  0.0723559
 ⋮                           

### TODO: Compute the risk-measure $\beta$-dictionary
Fill me in

In [16]:
β = Dict{String, Float64}();
for i ∈ 1:K
    ticker = my_tickers[i];
    σᵢ = Σ[i,index_market];
    σₘ = Σ[index_market,index_market]
    β[ticker] = σᵢ/σₘ;
end
β

Dict{String, Float64} with 29 entries:
  "JPM"   => 1.17336
  "MSFT"  => 1.13379
  "C"     => 1.48297
  "MRK"   => 0.504918
  "UNH"   => 0.878591
  "SPY"   => 1.0
  "BAC"   => 1.32778
  "MU"    => 1.52694
  "SRE"   => 0.794775
  "TSLA"  => 1.77511
  "KR"    => 0.194428
  "AMD"   => 1.67809
  "INTC"  => 1.12753
  "NVDA"  => 1.78828
  "MMM"   => 0.938743
  "NOC"   => 0.566008
  "ECL"   => 1.13742
  "LMT"   => 0.67081
  "GS"    => 1.26915
  "OXY"   => 1.54752
  "CMCSA" => 0.856908
  "JNJ"   => 0.548523
  "SPYD"  => 1.03756
  "NFLX"  => 1.07208
  "WFC"   => 1.25441
  ⋮       => ⋮

## Task 2:  Build risk-aware ticker-picker agent and world model
Fill me in

### TODO: Get something from somewhere
File me in

In [19]:
bandit_model = saved_state_dict["banditmodel"];

### TODO: Setup a MyTickerPickerRiskAwareWorldModel instance
Now that we have set up the bandit model, we construct [a `MyTickerPickerRiskAwareWorldModel` instance](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.MyTickerPickerRiskAwareWorldModel) which holds data about the world that we sample. The `world(...)` function takes problem data and returns a binary reward. First, we formulate the `world(...)` function, which takes the `start::Int64` index, an action $a\in\mathcal{A}$ value, and the `worldmodel::MyTickerPickerWorldModel.` The `world(...)` function returns a binary reward $r\in\left\{0,1\right\}$.

#### Summary of the risk-award world function
`Unhide` the code block to see a reference implementation of the `world(...)` function. 
* We compare the wealth generated by investing `1 USD` in a fund that returns the risk-free rate for `buffersize::Int64` periods of $\Delta{t}$ duration (denoted as `W_risk_free`) versus `1 USD` in a `ticker` over the same period (denoted as `W_ticker`). If `W_ticker > W_risk_free`, the `world(...)` function returns `1`, i.e., success; otherwise, the `world(...)` function returns `0.`

In [21]:
function world(start::Int64, action::Int64, worldmodel::MyTickerPickerRiskAwareWorldModel)::Int64
    
    # initialize 
    tickers = worldmodel.tickers;
    data = worldmodel.data;
    risk_free_rate = worldmodel.risk_free_rate;
    Δt = worldmodel.Δt;
    buffersize = worldmodel.buffersize;
    risk_dictionary = worldmodel.risk;
    
    # compute the expected return over the horizon -
    ticker_symbol = tickers[action]; # ticker we are looking at
    price_df = data[ticker_symbol];
    time = range(start+1,(start+buffersize), step = 1) |> collect
    
    buffer = Array{Float64,1}();
    for t ∈ time
        P₁ = price_df[t-1, :volume_weighted_average_price]
        P₂ = price_df[t, :volume_weighted_average_price]
        R = (1/Δt)*log(P₂/P₁); # growth rate
        push!(buffer,R);
    end
    β = risk_dictionary[ticker_symbol];
    sum_value = sum(buffer); # normalize by the risk
    μ = (β^(-1*buffersize*sign(sum_value)))*sum_value;

    # if we invested 1 USD, in each find how much would we have at the end of the horizon -
    W_risk_free = exp(buffersize*risk_free_rate*Δt); # only risk-free assets
    W_ticker = exp(μ*Δt);

    # Are we better or worse relative to the risk-free investment?
    result_flag = nothing;
    (W_ticker > W_risk_free) ? result_flag = 1 : result_flag = 0;

    # @show (W_risk_free, W_ticker, ticker_symbol, start, result_flag)
    
    return result_flag;
end;

Next, you need to set the `buffersize::Int64` variable. This variable controls the number of trading periods of duration $\Delta{t}$ that we use to compute the return.
* __Note__: The reference implementation above uses overlapping evaluation periods, not discrete contiguous periods. A more realistic implementation would refactor the reference implementation to use non-overlapping periods.

In [23]:
buffersize = 5; # 1-trading week

Finally, we construct a [`MyTickerPickerWorldModel` instance](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.MyTickerPickerWorldModel) using 
[a custom `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.build-Tuple{Type{MyTickerPickerWorldModel},%20NamedTuple}). The [`build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.build-Tuple{Type{MyTickerPickerWorldModel},%20NamedTuple}) take two arguments: first, the [`MyTickerPickerWorldModel` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.MyTickerPickerWorldModel) and the second is a [Julia `NamedTuple` instance](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) holding data for the model:
* The [`MyTickerPickerWorldModel` instance](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.MyTickerPickerWorldModel) requires the `my_tickers::Array{String,1}` array, the `data::Dict{String, DataFrame}` field holding the price data dictionary, the `world::Function`, the `risk_free_rate::Float64`, the duration of the time-step $\Delta{t}$, and the `buffersize::Int64` field. We store the world model in the `world_model::MyTickerPickerWorldModel` variable:

In [25]:
risk_aware_world_model = build(MyTickerPickerRiskAwareWorldModel, (
    tickers = my_tickers,
    data = dataset,
    world = world,
    risk_free_rate = risk_free_rate,
    Δt = Δt,
    buffersize = buffersize,
    risk = β
));

## Task 3: Run a collection of risk-aware ticker-picker agents and examine their preferences
In this task, explore the preference of `N` agents by running the [`sample(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.sample-Tuple{MyEpsilonSamplingBanditModel,%20MyTickerPickerWorldModel}) inside a [`for-loop`](https://docs.julialang.org/en/v1/base/base/#for). We'll store the results at a particular `trading_day_index::Int64` in the `agent_specific_data::Array{Beta,2}(undef, K, number_of_agents)` array. 
* The `agent_specific_data` array holds the [`Beta` distributions](https://juliastats.org/Distributions.jl/stable/univariate/#Distributions.Beta) for each agent at `trading_day_index::Int64` for each `ticker`$\in$`tickers.` The `ticker` is the row index, and the agent's [`Beta` distributions](https://juliastats.org/Distributions.jl/stable/univariate/#Distributions.Beta) is the column index, e.g., index (1,4) holds the preference for ticker = 1 held by agent = 4 at time `trading_day_index::Int64.`
* In the code block below, we use [the Julia `foreach(...)` iteration pattern](https://docs.julialang.org/en/v1/base/collections/#Base.foreach) to populate the `agent_specific_data` array. We could have also used [a simple `for-loop`](https://docs.julialang.org/en/v1/base/base/#for), but the [`foreach(...)` iteration pattern](https://docs.julialang.org/en/v1/base/collections/#Base.foreach) is more compact.

In [27]:
number_of_agents = 5000; # how many agents do we want
trading_day_index = (maximum_number_trading_days - buffersize); # What trading day are we interested in?
risk_aware_agent_specific_data = Array{Beta,2}(undef, K, number_of_agents);

for agent_index ∈ 1:number_of_agents
    
    # sample -
    time_sample_results_dict_eps = VLQuantitativeFinancePackage.sample(bandit_model, risk_aware_world_model; 
        horizon = (maximum_number_trading_days - buffersize));

    # populate the agent_specific_data array
    foreach(k-> risk_aware_agent_specific_data[k, agent_index] = time_sample_results_dict_eps[trading_day_index][k], 1:K)
end

In [28]:
risk_aware_agent_specific_data[:,1]

29-element Vector{Beta}:
 Beta{Float64}(α=23.0, β=22.0)
 Beta{Float64}(α=7.0, β=10.0)
 Beta{Float64}(α=15.0, β=15.0)
 Beta{Float64}(α=11.0, β=13.0)
 Beta{Float64}(α=12.0, β=13.0)
 Beta{Float64}(α=26.0, β=21.0)
 Beta{Float64}(α=9.0, β=13.0)
 Beta{Float64}(α=7.0, β=11.0)
 Beta{Float64}(α=7.0, β=12.0)
 Beta{Float64}(α=3.0, β=8.0)
 Beta{Float64}(α=19.0, β=21.0)
 Beta{Float64}(α=45.0, β=38.0)
 Beta{Float64}(α=17.0, β=18.0)
 ⋮
 Beta{Float64}(α=3.0, β=8.0)
 Beta{Float64}(α=41.0, β=32.0)
 Beta{Float64}(α=7.0, β=13.0)
 Beta{Float64}(α=16.0, β=18.0)
 Beta{Float64}(α=16.0, β=17.0)
 Beta{Float64}(α=25.0, β=22.0)
 Beta{Float64}(α=67.0, β=47.0)
 Beta{Float64}(α=28.0, β=21.0)
 Beta{Float64}(α=32.0, β=25.0)
 Beta{Float64}(α=9.0, β=13.0)
 Beta{Float64}(α=7.0, β=11.0)
 Beta{Float64}(α=4.0, β=10.0)

### TODO: Compute the wisdom of the risk-aware collective
Now that we have preferences for the `N` agents (encoded as `Beta` distributions for each ticker), let's develop a consensus belief in which tickers we might include in our portfolio $\mathcal{P}$. First, compute the agent-specific rank of each `ticker`$\in$`my_tickers,` where `rank = 1` is the best, and `rank = K` is the worst. We'll store these values in the `preference_rank_array::Array{Int,2}` array.

In [30]:
risk_aware_preference_rank_array = Array{Int,2}(undef, number_of_agents, K);
for agent ∈ 1:number_of_agents
        
    # ask an agent about their preference for ticker i -
    experience_distributions = risk_aware_agent_specific_data[:,agent]
    preference_vector = preference(experience_distributions, my_tickers) .|> x-> trunc(Int64, x) # wow the trunc function is cool!

    # package
    foreach(i -> risk_aware_preference_rank_array[agent, i] = preference_vector[i], 1:K);
end
risk_aware_preference_rank_array

5000×29 Matrix{Int64}:
 11  20  12  19  16   5  21  23  25  …  18  15   9   1   2   4  21  23  27
 23   7  10  17  11   5  21   9   2     15  20  28   3   4  12  28  17  19
 27   8  21  22   6   1  11  17  15     25  12   5  19  19  23  29   7  19
  5  21   3  19  14  22  20  10  15     13  27   7  12   2   8  25   8  17
  2  25  16  27   3  19   4  15  16      6   9  22  23  28  13  18   8  10
  5  23  18  13   3  19  28  11   4  …   8   2   6   1  23   9  16  14  28
 14  21  12  16   4   8  15  24  24     22   6   5   2  10  27  16  29   7
 15  28  11  22  21  15  26   5  18     10   4   2  15   3   8  29   7  15
 18  25  25  11   5  28  16  10  15     17  19  23  24   7  22  14   3  12
  7  29  20  21  14   1  16  19  25     26   6  11  10   9  28  22  15  13
  2  26  17  24  19  17  21   9  22  …  25   7   1  14  15  11  27  17  23
  4  23   6  21  19  26  26  10  15     28  29   1  12  16   9  21   3  11
  7   9  21  17  10   1   6  20   8     29  14   5  12  25   2  28  11   4
  

The rows of the `risk_aware_preference_rank_array::Array{Int,2}` correspond to agents, while the columns hold each firm's ranking by that agent, with smaller values indicating a higher preference. The agent ranking was calculated [using the `preference(...)` function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/bandits/#VLQuantitativeFinancePackage.preference) which agent's belief (encoded as an array of [`Beta` distributions](https://juliastats.org/Distributions.jl/stable/univariate/#Distributions.Beta)) and returns the ordinal ranking of each `ticker`$\in$`my_tickers.`

#### Compute the risk aware probability dictionary
Let's count the number of times a `ticker` is ranked in the `top M` tickers across the `N` agents and then normalize by the number of agents, i.e., estimate the probability of being ranked in the `top M` using a [`for-loop`](https://docs.julialang.org/en/v1/base/base/#for).
* First, we set a value `M::Int64`, the number of top firms we consider. Then, we initialize data storage for the probability values. We'll store the probability value in the `probability_top_dictionary::Dict{String, Float64},` where the `ticker::String` is the key, and the frequency is the value.
* For ticker index `i`, we first compute the probability, and then we store the probability in the `risk_aware_probability_top_dictionary::Dict{String, Float64}`. In this code, we make heavy use of the [Julia pipe `|>` operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping) to condense many operations into a single line. Yes, that is not great for readability, but wow, what a Flex!

In [33]:
M = 10; # look for top M firms
risk_aware_probability_top_dictionary = Dict{String,Float64}();
for i ∈ eachindex(my_tickers)
    probability = findall(x-> x ≤ M, risk_aware_preference_rank_array[:,i]) |> x -> length(x) |> x-> x/number_of_agents; # compute probability
    my_tickers[i] |> ticker -> risk_aware_probability_top_dictionary[ticker] = probability; # store probability
end

### TODO: Build a preference table over the population of agents for risk blind and risk-aware agents
The `probability_top_dictionary::Dict{String, Float64}` holds the probability that a `ticker`$\in$ `my_tickers` is ranked in the `top M` firms. Let's now look at the ordinal ranking and propose some tests of our interpretation of this data.

In [35]:
let
    # initialize 
    df = DataFrame();
    tmp_risk_blind = Array{Float64,1}();
    tmp_risk_aware = Array{Float64,1}();

    # compute the ordinal rank of the sorted list of tickers
    my_sorted_tickers = my_tickers |> sort;

    # blind -
    foreach(ticker -> push!(tmp_risk_blind, risk_blind_probability_top_dictionary[ticker]), my_sorted_tickers)
    ordinal_rank_risk_blind = ordinalrank(tmp_risk_blind, rev=true); # function exported by StatsBase.jl

    # aware -
    foreach(ticker -> push!(tmp_risk_aware, risk_aware_probability_top_dictionary[ticker]), my_sorted_tickers)
    ordinal_rank_risk_aware = ordinalrank(tmp_risk_aware, rev=true); # function exported by StatsBase.jl
    
    # populate rows in the df -
    for i ∈ eachindex(my_sorted_tickers)
        ticker_value = my_sorted_tickers[i];    
        freq_value = 
        row_df = (
            ticker = ticker_value,
            β = β[ticker_value],
            probability_risk_blind = risk_blind_probability_top_dictionary[ticker_value],
            probability_risk_aware = risk_aware_probability_top_dictionary[ticker_value],
            rank_risk_blind = ordinal_rank_risk_blind[i],
            rank_risk_aware = ordinal_rank_risk_aware[i],
            Δ = (ordinal_rank_risk_aware[i] - ordinal_rank_risk_blind[i])
        );
        push!(df, row_df);
    end
    pretty_table(df); 
end

┌────────┬──────────┬────────────────────────┬────────────────────────┬─────────────────┬─────────────────┬───────┐
│ ticker │        β │ probability_risk_blind │ probability_risk_aware │ rank_risk_blind │ rank_risk_aware │     Δ │
│ String │  Float64 │                Float64 │                Float64 │           Int64 │           Int64 │ Int64 │
├────────┼──────────┼────────────────────────┼────────────────────────┼─────────────────┼─────────────────┼───────┤
│   AAPL │  1.24019 │                  0.544 │                 0.5478 │               4 │               4 │     0 │
│    AMD │  1.67809 │                  0.461 │                 0.1726 │               5 │              24 │    19 │
│    BAC │  1.32778 │                  0.283 │                 0.2206 │              21 │              23 │     2 │
│      C │  1.48297 │                 0.2734 │                 0.1232 │              23 │              27 │     4 │
│  CMCSA │ 0.856908 │                  0.288 │                 0.3696 │ 

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.